In [1]:
from python_on_whales import DockerClient, docker
import uuid

docker_clients = []
for i in range(3):
    worker_id = uuid.uuid4().hex
    docker_client = DockerClient(
        compose_project_name=worker_id,
        compose_files=["client-compose.yml"],
    )
    docker_clients.append(docker_client)



In [2]:
import os

for docker_client in docker_clients:
    os.environ["WORKER_ID"] = docker_client.compose.client_config.compose_project_name
    docker_client.compose.up(
        detach=True,
        build=True,
    )

 Network 83e6e2854e2f4e00bb6feb4755fdbd67_default  Creating
 Network 83e6e2854e2f4e00bb6feb4755fdbd67_default  Created
 Container 83e6e2854e2f4e00bb6feb4755fdbd67-app-account-1  Creating
 Container 83e6e2854e2f4e00bb6feb4755fdbd67-app-product-1  Creating
 Container 83e6e2854e2f4e00bb6feb4755fdbd67-app-product-1  Created
 Container 83e6e2854e2f4e00bb6feb4755fdbd67-app-account-1  Created
 Container 83e6e2854e2f4e00bb6feb4755fdbd67-app-product-1  Starting
 Container 83e6e2854e2f4e00bb6feb4755fdbd67-app-account-1  Starting
 Container 83e6e2854e2f4e00bb6feb4755fdbd67-app-product-1  Started
 Container 83e6e2854e2f4e00bb6feb4755fdbd67-app-account-1  Started
 Network 0338cb2acecb4edd849e7688e1b130c2_default  Creating
 Network 0338cb2acecb4edd849e7688e1b130c2_default  Created
 Container 0338cb2acecb4edd849e7688e1b130c2-app-account-1  Creating
 Container 0338cb2acecb4edd849e7688e1b130c2-app-product-1  Creating
 Container 0338cb2acecb4edd849e7688e1b130c2-app-account-1  Created
 Container 0338cb2a

In [8]:
docker.image.list()

[python_on_whales.Image(id='sha256:b4201f2066453', tags=['tiangolo/uvicorn-gunicorn-fastapi:latest']),
 python_on_whales.Image(id='sha256:a61b645b609b7', tags=['golang:latest'])]

In [5]:
for docker_client in docker_clients:
    docker_client.compose.down()

 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Stopping
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Stopping
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Stopped
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Removing
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Stopped
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Removing
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Removed
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Removed
 Network 577f83d86ebd46d59dcdd5c466efb96f_default  Removing
 Network 577f83d86ebd46d59dcdd5c466efb96f_default  Removed
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Stopping
 Container 9bae1087874f45d885f2d86ae6fe903f-app-account-1  Stopping
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Stopped
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Removing
 Container 9bae1087874f45d885f2d86ae6fe903f-app-account-1  Stopped
 Co

In [2]:
docker_clients[0].compose.ps()[0].network_settings.networks

NameError: name 'docker_clients' is not defined

In [6]:
for docker_client in docker_clients:
    for container in docker_client.compose.ps():
        A_WORKER_ID = docker_client.client_config.compose_project_name
        B_WORKER_ID = [env for env in container.config.env if "WORKER_ID" in env][0].split("=")[1]
        print(A_WORKER_ID, B_WORKER_ID, A_WORKER_ID == B_WORKER_ID)

d7d6146b1c2c4fb8ac4934277a567520 d7d6146b1c2c4fb8ac4934277a567520 True
d7d6146b1c2c4fb8ac4934277a567520 d7d6146b1c2c4fb8ac4934277a567520 True
67695a0fa1d64633bb89491ee632c256 67695a0fa1d64633bb89491ee632c256 True
67695a0fa1d64633bb89491ee632c256 67695a0fa1d64633bb89491ee632c256 True
5ddd1e567017446abf33dbc07911c8a2 5ddd1e567017446abf33dbc07911c8a2 True
5ddd1e567017446abf33dbc07911c8a2 5ddd1e567017446abf33dbc07911c8a2 True


In [5]:
from httpx import Client

http_client = Client(
    base_url="http://172.18.0.3:8000",
)
http_client.get("/").json()

{'worker_id': 'd7d6146b1c2c4fb8ac4934277a567520', 'app_name': 'app-1'}

In [1]:
import autocode

optimization = autocode.Optimization(
    num_cpus=32,
    server_host="0.0.0.0",
    server_port=10000,
    dashboard_port=10001,
)

2024-08-03 02:06:53,970	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at http://172.25.4.192:8265 


Available resources:  {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'node:172.25.4.192': 1.0, 'CPU': 32.0, 'memory': 9135476736.0, 'object_store_memory': 4567738368.0, 'GPU': 1.0}
Server is healthy.


In [2]:
optimization.reset()

In [4]:
from autocode import OptimizationObjective
from typing import Dict, Any, List
from autocode import OptimizationEvaluateRunResponse

objectives: List[OptimizationObjective] = [
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),

    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
]


def evaluator(input: List[OptimizationEvaluateRunResponse]) -> Dict[str, Any]:
    output: Dict[str, Any] = {
        "F": input[0].objectives + input[1].objectives,
    }

    return output


In [5]:
result = optimization.run(
    objectives=objectives,
    num_inequality_constraints=0,
    num_equality_constraints=0,
    evaluator=evaluator
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |     93 |             - |             -
     2 |      200 |    100 |  1.2892156863 |         nadir
     3 |      300 |    100 |  0.5338345865 |         nadir
     4 |      400 |    100 |  2.1029411765 |         nadir
     5 |      500 |    100 |  0.4848484848 |         nadir
     6 |      600 |    100 |  0.3720930233 |         nadir
     7 |      700 |    100 |  0.2954545455 |         nadir
     8 |      800 |    100 |  0.7773109244 |         nadir
     9 |      900 |    100 |  0.5150375940 |         nadir
    10 |     1000 |    100 |  3.4905660377 |         nadir
    11 |     1100 |    100 |  0.0591715976 |         nadir
    12 |     1200 |    100 |  1.7593582888 |         nadir
    13 |     1300 |    100 |  0.1407766990 |         nadir
    14 |     1400 |    100 |  0.0028985507 |         ideal
    15 |     1500 |    100 |  3.4973262032 |         nadir
    16 |     1600 |    100 |  0.0181818182 |         nad

Exception ignored in: <module 'threading' from '/home/muazhari/miniconda3/lib/python3.11/threading.py'>
Traceback (most recent call last):
  File "/home/muazhari/miniconda3/lib/python3.11/threading.py", line 1583, in _shutdown
    lock.acquire()
  File "/home/muazhari/miniconda3/lib/python3.11/site-packages/streamlit/web/bootstrap.py", line 44, in signal_handler
    server.stop()
  File "/home/muazhari/miniconda3/lib/python3.11/site-packages/streamlit/web/server/server.py", line 405, in stop
    self._runtime.stop()
  File "/home/muazhari/miniconda3/lib/python3.11/site-packages/streamlit/runtime/runtime.py", line 324, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/home/muazhari/miniconda3/lib/python3.11/asyncio/base_events.py", line 806, in call_soon_threadsafe
    self._check_closed()
  File "/home/muazhari/miniconda3/lib/python3.11/asyncio/base_events.py", line 519, in _check_closed
    raise RuntimeError('Event loop is closed')
RuntimeError: Event 

KeyboardInterrupt: 